In [ ]:
import pymysql
pymysql.install_as_MySQLdb()
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import ecco
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)

## 데이터베이스 연결
connect = pymysql.connect(host='database-skku.c6dzc5dnqf69.ap-northeast-2.rds.amazonaws.com', user='admin',
                          password='tjdrbsrhkseo123', db='public', charset='utf8mb4')
cursor = connect.cursor()
df = pd.read_sql_query('select * from medii.cris_dataset', connect)


# ecco language model
# lm = ecco.from_pretrained('distilgpt2', activations=True)
# test = df[:5]
# train = df[5:]
train = df
## trainset 벡터화

class StemmedTfidfVectorizer(TfidfVectorizer):
    en_stemmer = SnowballStemmer('english')
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer, self).build_analyzer()
        return lambda doc: (StemmedTfidfVectorizer.en_stemmer.stem(w) for w in analyzer(doc))

tfidf = StemmedTfidfVectorizer(
    analyzer="word", 
    stop_words='english', 
    ngram_range=(1,1), 
    min_df=1    # limit of minimum number of counts: 3
)



vectors = tfidf.fit_transform(train['study_summary_eng'].tolist())

print(vectors.shape)
print(vectors)
## NMF를 이용하여 본문에서 특성 추출(200개의 특성)
from sklearn.decomposition import NMF

vector_array = vectors.toarray()
nmf = NMF(n_components=90)
nmf.fit(vector_array)
features = nmf.transform(vector_array)

from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

print(norm_features[0:2])
# # testset 벡터화
# vectorizer = TfidfVectorizer(stop_words = 'english')
# test_vectors = vectorizer.fit_transform(test['study_summary_eng'].tolist())

# print(test_vectors.shape)
# #  


# test_vector_array = test_vectors.toarray()
# test_features = nmf.transform(test_vector_array)

# from sklearn.preprocessing import Normalizer

# normalizer = Normalizer()
# test_norm_features=normalizer.fit_transform(test_features)

# print(test_norm_features[0:2])


## df_defeatures 확인
df_features = pd.DataFrame(norm_features,index=train.index.tolist())

## 문서 유사도 계산 

for i in range(5):
    article = df_features.loc[i]
    similarities=df_features.dot(article)

    ## 유사도 높은 문서 추출
    top=similarities.nlargest(n=11)
    texts = train.loc[top.index]['main_title_kor'].tolist()

    print(top)

    for text in texts:
        print(text)
    print()